# Margins for SNP
-[ ] Build / Extract the chains    
-[ ] Get undPrices   
-[ ] Integrate undPrices to the chains   

-[ ] Get the lots   
-[ ] Get margins for with qty as the lot   
-[ ] Make logic for margins from SNP rules   
-[ ] For margins with NaN, replace margins with the one derived from logic   
-[ ] Integrate margins to chains

In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_async import util, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER
COLS = _vars.COLS[0]
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Suppress Errors
util.logToFile(DATAPATH.parent.parent / 'log' / 'ztest.log')

In [ ]:
# Build chains
from utils import make_chains_with_margins, get_strike_closest_to_und
df_chains = make_chains_with_margins(MARKET=MARKET)
# df = get_strike_closest_to_und(df_chains)

In [ ]:
# Make raw targets from the chains
from utils import raw_target_opts
df_ch = raw_target_opts(df_chains)
df_ch.head()

In [5]:
# qualify the targets
from utils import qualify_naked_opts, pickle_me, get_pickle, nakeds_with_expPrice
df_naked_targets = qualify_naked_opts(df_ch)

pickle_me(df_naked_targets, DATAPATH / 'df_naked_targets.pkl')
df = get_pickle(DATAPATH / 'df_naked_targets.pkl')

In [ ]:
# build nakeds with expected Price
from utils import nakeds_with_expPrice
dfn = nakeds_with_expPrice(df)